In [1]:
# adapted from https://pyro.ai/examples/prodlda.html
# imports
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import torch
import time

import math
import torch.nn as nn
import torch.nn.functional as F
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, TraceMeanField_ELBO
from tqdm import trange

#plotting libraries!
import matplotlib.pyplot as plt
%matplotlib inline

# Import the wordcloud library
from wordcloud import WordCloud

from sklearn.metrics.pairwise import cosine_similarity

/opt/homebrew/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Apply to movie set**  
**end goal:**  
want to  map/group movies (doc id) to topics so we can look up a topic and find related movies or vice versa  
**what we have:**  
movie descriptions for ~9k movies from netflix titles database  
**conversion:**  
- our LDA model takes in a matrix of token counts and outputs beta values
- need to convert what we have into a matrix of token counts using the CountVectorizer function
    - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [2]:
# start timer
start_whole = time.perf_counter()

# import user rating data
users_df = pd.read_csv("./ml_netflix.csv")
eval_df = pd.read_csv("./ml_netflix.csv")
eval_df.drop_duplicates('title', inplace=True)
eval_df.drop(columns=['userId', 'rating'], inplace=True)

num_movies = users_df['movieId'].nunique()
num_users = num_users = users_df['userId'].nunique()

In [3]:
data = eval_df['description'].to_numpy() 
# `max_df` removes common words - words with frequency higher than this across all documents
# `min_df` removes rare words - words that appear in less than min_df number of docs
# note: if min_df is too big, will get error in word cloud (i think because not enough data for all topics)
# max min_df i've found that works is 5
vectorizer = CountVectorizer(min_df=5, max_df=0.7, stop_words='english') # 1, 0.7
# docs is a binary matrix of size M x N where M is number of movies, N is number of words in vocab
# docs[i, j] is 1 if movie i's descriptoin has word j and 0 otherwise
docs = torch.from_numpy(vectorizer.fit_transform(data).toarray())

# creates df with words and indices corresponding to words 
# used to convert betas back to words in word cloud plots
vocab = pd.DataFrame(columns=['word', 'index'])
vocab['word'] = vectorizer.get_feature_names()
vocab['index'] = vocab.index

print('Dictionary size: %d' % len(vocab))
print('Corpus size: {}'.format(docs.shape))

Dictionary size: 639
Corpus size: torch.Size([1056, 639])


/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [4]:
# setting global variables related to LDA computation
smoke_test=False
seed = 0
torch.manual_seed(seed)
pyro.set_rng_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_topics = 25 if not smoke_test else 3
docs = docs.float().to(device)
batch_size = 32
learning_rate = 1e-3
num_epochs = 50 if not smoke_test else 1

In [5]:
class Encoder(nn.Module):
    # Base class for the encoder net, used in the guide
    def __init__(self, vocab_size, num_topics, hidden, dropout):
        super().__init__()
        self.drop = nn.Dropout(dropout)  # to avoid component collapse
        self.fc1 = nn.Linear(vocab_size, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fcmu = nn.Linear(hidden, num_topics)
        self.fclv = nn.Linear(hidden, num_topics)
        # NB: here we set `affine=False` to reduce the number of learning parameters
        # See https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html
        # for the effect of this flag in BatchNorm1d
        self.bnmu = nn.BatchNorm1d(num_topics, affine=False)  # to avoid component collapse
        self.bnlv = nn.BatchNorm1d(num_topics, affine=False)  # to avoid component collapse

    def forward(self, inputs):
        h = F.softplus(self.fc1(inputs))
        h = F.softplus(self.fc2(h))
        h = self.drop(h)
        # μ and Σ are the outputs
        logtheta_loc = self.bnmu(self.fcmu(h))
        logtheta_logvar = self.bnlv(self.fclv(h))
        logtheta_scale = (0.5 * logtheta_logvar).exp()  # Enforces positivity
        return logtheta_loc, logtheta_scale


class Decoder(nn.Module):
    # Base class for the decoder net, used in the model
    def __init__(self, vocab_size, num_topics, dropout):
        super().__init__()
        self.beta = nn.Linear(num_topics, vocab_size, bias=False)
        self.bn = nn.BatchNorm1d(vocab_size, affine=False)
        self.drop = nn.Dropout(dropout)

    def forward(self, inputs):
        inputs = self.drop(inputs)
        # the output is σ(βθ)
        return F.softmax(self.bn(self.beta(inputs)), dim=1)


class ProdLDA(nn.Module):
    def __init__(self, vocab_size, num_topics, hidden, dropout):
        super().__init__()
        self.vocab_size = vocab_size
        self.num_topics = num_topics
        self.encoder = Encoder(vocab_size, num_topics, hidden, dropout)
        self.decoder = Decoder(vocab_size, num_topics, dropout)

    def model(self, docs):
        pyro.module("decoder", self.decoder)
        with pyro.plate("documents", docs.shape[0]):
            # Dirichlet prior 𝑝(𝜃|𝛼) is replaced by a logistic-normal distribution
            logtheta_loc = docs.new_zeros((docs.shape[0], self.num_topics))
            logtheta_scale = docs.new_ones((docs.shape[0], self.num_topics))
            logtheta = pyro.sample(
                "logtheta", dist.Normal(logtheta_loc, logtheta_scale).to_event(1))
            theta = F.softmax(logtheta, -1)

            # conditional distribution of 𝑤𝑛 is defined as
            # 𝑤𝑛|𝛽,𝜃 ~ Categorical(𝜎(𝛽𝜃))
            count_param = self.decoder(theta)
            # Currently, PyTorch Multinomial requires `total_count` to be homogeneous.
            # Because the numbers of words across documents can vary,
            # we will use the maximum count accross documents here.
            # This does not affect the result because Multinomial.log_prob does
            # not require `total_count` to evaluate the log probability.
            total_count = int(docs.sum(-1).max())
            pyro.sample(
                'obs',
                dist.Multinomial(total_count, count_param),
                obs=docs
            )

    def guide(self, docs):
        pyro.module("encoder", self.encoder)
        with pyro.plate("documents", docs.shape[0]):
            # Dirichlet prior 𝑝(𝜃|𝛼) is replaced by a logistic-normal distribution,
            # where μ and Σ are the encoder network outputs
            logtheta_loc, logtheta_scale = self.encoder(docs)
            logtheta = pyro.sample(
                "logtheta", dist.Normal(logtheta_loc, logtheta_scale).to_event(1))

    def beta(self):
        # beta matrix elements are the weights of the FC layer on the decoder
        return self.decoder.beta.weight.cpu().detach().T

In [6]:
# training 
pyro.clear_param_store()

prodLDA = ProdLDA(
    vocab_size=docs.shape[1],
    num_topics=num_topics,
    hidden=100 if not smoke_test else 10,
    dropout=0.2
)
prodLDA.to(device)

optimizer = pyro.optim.Adam({"lr": learning_rate})
svi = SVI(prodLDA.model, prodLDA.guide, optimizer, loss=TraceMeanField_ELBO())
num_batches = int(math.ceil(docs.shape[0] / batch_size)) if not smoke_test else 1

start = time.perf_counter()
bar = trange(num_epochs)
for epoch in bar:
    running_loss = 0.0
    for i in range(num_batches):
        batch_docs = docs[i * batch_size:(i + 1) * batch_size, :]
        loss = svi.step(batch_docs)
        running_loss += loss / batch_docs.size(0)

    bar.set_postfix(epoch_loss='{:.2e}'.format(running_loss))
end = time.perf_counter()
print(f"Finished in {(end - start)/60:0.4f} minutes")

100%|██████████| 50/50 [00:05<00:00,  9.95it/s, epoch_loss=1.60e+03]

Finished in 0.0839 minutes


In [7]:
def plot_word_cloud(b, ax, v, n):
    sorted_, indices = torch.sort(b, descending=True) # sorts betas by descending order
    df = pd.DataFrame(indices[:100].numpy(), columns=['index'])
    words = pd.merge(df, v[['index', 'word']],
                     how='left', on='index')['word'].values.tolist()
    sizes = (sorted_[:100] * 1000).int().numpy().tolist()
    freqs = {words[i]: sizes[i] for i in range(len(words))}
    wc = WordCloud(background_color="white", width=800, height=500)
    wc = wc.generate_from_frequencies(freqs)
    ax.set_title('Topic %d' % (n + 1))
    ax.imshow(wc, interpolation='bilinear')
    ax.axis("off")

# beta = prodLDA.beta()
# fig, axs = plt.subplots(7, 3, figsize=(14, 24))
# for n in range(beta.shape[0]):
#     i, j = divmod(n, 3)
#     plot_word_cloud(beta[n], axs[i, j], vocab, n)
# axs[-1, -1].axis('off');

# plt.show()

**adding topic number and probability of being in topic to netflix db**
- for each movie, need to find largest topic probability based on words in docs corpus array

In [8]:
# betas is a TxW array where T is number of topics, W is number of words
# betas[i, j] gives probability that word j is in topic i
betas = prodLDA.beta()
# for each movie, find maximum of beta values to assign topic
topics = []
topic_probs = []
for i in range(num_movies):
    words_tensor = docs[i]
    word_indices = ((words_tensor==1).nonzero(as_tuple=True)[0]) # get indices of words used in movie eval_df[i]
    max_probability = 0
    topic = None
    for word_index in word_indices: # find max topic prob given words in movie i from betas object
        max_word_probability = max(betas[:,word_index]).numpy()
        if max_word_probability > max_probability:
            max_probability = max_word_probability
            topic = torch.argmax(betas[:,word_index]).numpy()+1
    topics.append(topic)
    topic_probs.append(max_probability)
eval_df['Topic'] = topics
eval_df['Probability'] = topic_probs
eval_df['Doc'] = range(num_movies)

eval_df.head()

,title,description,movieId,Topic,Probability,Doc
0,Grown Ups,Mourning the loss of their beloved junior high...,0,18,0.22780892,0
10,Dark Skies,A familys idyllic suburban life shatters when ...,1,6,0.29835275,1
12,Jaws,When an insatiable great white shark terrorize...,2,13,0.23815896,2
103,Jaws 2,Four years after the last deadly shark attacks...,3,13,0.23815896,3
120,Jaws: The Revenge,"After another deadly shark attack, Ellen Brody...",4,12,0.21340157,4


In [9]:
def recommend_by_storyline(title, df): # per jessica's code
    recommended = []
    top10_list = []
    
    title = title.lower()
    df['title'] = df['title'].str.lower()
    topic_num = df[df['title']==title].Topic.values
    if len(topic_num) == 0:
        print(title + " not in database")
        return
    doc_num = df[df['title']==title].Doc.values    
    
    output_df = df[df['Topic']==topic_num[0]].sort_values('Probability', ascending=False).reset_index(drop=True)

    index = output_df[output_df['Doc']==doc_num[0]].index[0]
    
    # return the 10 results with closest probability of belonging to topic `topic_num`
    top10_list += list(output_df.iloc[index-5:index].index)
    top10_list += list(output_df.iloc[index+1:index+6].index)
    
    output_df['title'] = output_df['title'].str.title()
    for each in top10_list:
        recommended.append(output_df.iloc[each].title)
        
    return recommended

In [10]:
recommend_by_storyline("Jaws", eval_df)

['The Witch',
 'The Tuxedo',
 'The Runner',
 'The Mist',
 'Rebel Without A Cause',
 'Red',
 'Cop Car',
 'Jaws 2',
 'Blue Streak',
 'Vir Das: Abroad Understanding']

now, we have a way to recommend movies given a movie, now we move to **evaluation** of our model
- **MAP@K**
- **MAR@K**
- **coverage**: percent of items in the traning data the movie is able to recommend on a test set
- **personalization**: dissimilarity (1 - cosine similarity) between user's lists of recommendations

using movielens dataset of user ratings in order to evaluate based on our top 10 movies for our model?
in small movielens db, we have 1119 possible movies

let's say that the movies we recommend to each user are the top 10 in the topic that they have the most
5.0 ratings in. if no 5.0 ratings, ..

relevant links:  
https://towardsdatascience.com/evaluation-metrics-for-recommender-systems-df56c6611093  
http://sdsawtelle.github.io/blog/output/mean-average-precision-MAP-for-recommender-systems.html

In [11]:
# add topics to users_df
users_df['Topic'] = 0
users_df.dropna(inplace=True)

for index, row in users_df.iterrows():
    users_df.at[index, 'Topic'] = eval_df[eval_df['movieId'] == row['movieId']]['Topic'].iloc[0]
    users_df.at[index, 'Probability'] = eval_df[eval_df['movieId'] == row['movieId']]['Probability'].iloc[0]

users_df

,title,description,movieId,userId,rating,Topic,Probability
0,Grown Ups,Mourning the loss of their beloved junior high...,0,0,2.0,18,0.227809
1,Grown Ups,Mourning the loss of their beloved junior high...,0,1,3.5,18,0.227809
2,Grown Ups,Mourning the loss of their beloved junior high...,0,2,5.0,18,0.227809
3,Grown Ups,Mourning the loss of their beloved junior high...,0,3,3.0,18,0.227809
4,Grown Ups,Mourning the loss of their beloved junior high...,0,4,2.5,18,0.227809
...,...,...,...,...,...,...,...
17557,Zombieland,Looking to survive in a world taken over by zo...,1054,539,3.0,10,0.200234
17558,Zombieland,Looking to survive in a world taken over by zo...,1054,296,3.5,10,0.200234
17559,Zombieland,Looking to survive in a world taken over by zo...,1054,95,3.0,10,0.200234
17560,Zombieland,Looking to survive in a world taken over by zo...,1054,11,3.5,10,0.200234


In [12]:
# for each topic, precompute sorted probabilities
per_topic = [] # list of dfs

for topic in range(1, num_topics+1):
    output_df = eval_df[eval_df['Topic']==topic].sort_values('Probability', ascending=False).reset_index(drop=True)
    per_topic.append(output_df)

In [13]:
# calculate recommendations for each user
"""
as suggested in project proposal, number of recommendations k depends on user
we set k = minimum(number of movies the user has given 5 stars, number of movies the user has given 4 stars)

we calculate recommendation by taking all movies that the user rated 5 stars (or 4 stars if no 5 stars)
then, for each movie, we find the topic it is in by finding the topic that it has the highest probability in
and find the movie in that topic with probability closest to that movie's topic probabiity
"""

# find all 4 and 5 star ratings
user_5_df = users_df[users_df['rating']==5]
user_4_df = users_df[users_df['rating']==4]

# for each user, calculate recommended movies
# recommendations is 2d array where recommendations[i] is a list with first value = userId, 
# subsequent values are k recommended movieIds for the user 
# where k is number of 5 star ratings given by user and if no 5 star ratings, number of 4 star ratings given by user
recommendations = []
for user in range(num_users):
    # find number of 5 ratings
    temp_df = user_5_df[user_5_df['userId']==user]
    # if no 5 ratings, find number of 4 ratings
    if temp_df.empty:
        temp_df = user_4_df[user_4_df['userId']==user]
    # if no 4 ratings, go to next user
    # if temp_df.empty:
    #     continue
    recommendations.append([user])
    # for each movie
    for index, row in temp_df.iterrows():
        topic = row['Topic']
        movieId = row['movieId']
        prob = row['Probability']
        topic_df = per_topic[topic-1]
        # find closest probability in topic to the current movie
        index = topic_df[topic_df['movieId']==movieId].index[0]
        # accounting for out of bound accessing
        if index == 0:
            recommendations[-1].append(row_below['movieId'])
            continue
        if index == len(topic_df)-1:
            recommendations[-1].append(row_above['movieId'])
            continue    
        row_above = topic_df.iloc[index-1]
        row_below = topic_df.iloc[index+1]
        if row_below['Probability'] - prob < row_above['Probability'] - prob:
            recommendations[-1].append(row_below['movieId'])
        else:
            recommendations[-1].append(row_above['movieId'])

print(len(recommendations))


608


In [14]:
# calculate personalization matrix
# https://towardsdatascience.com/evaluation-metrics-for-recommender-systems-df56c6611093
# create num_user x num_movies
num_users = len(recommendations)
personalization = np.zeros((num_users, num_movies))

for row in recommendations:
    user = row[0]
    for movie_ind in row[1:]:
        personalization[user][movie_ind] = 1

cosine_sim = cosine_similarity(personalization, personalization)

# compute average of upper triangle to get cosine similarity
sum = 0
denom = (num_users-1)*(num_users)/2
for i in range(num_users-1):
    for j in range(i+1, num_users):
        sum += cosine_sim[i][j]

similarity = sum/denom
dissimilarity = 1 - similarity
print(dissimilarity)

0.9458428843171724


In [15]:
# function to calculate MAP @ k or MAR @ k
# http://sdsawtelle.github.io/blog/output/mean-average-precision-MAP-for-recommender-systems.html

# map = true if we wish to calculate precision, false if we wish to calculate recall
def map_or_mark(recommendations, num_users, user_5_df, user_4_df, map):
    # for each user
    map_k = 0
    for user in range(num_users):
        l = len(recommendations[user])
        user_actual = user_5_df[user_5_df['userId']==user]['movieId']
        if user_actual.empty:
            user_actual = user_4_df[user_4_df['userId']==user]
        sum = 0
        for k in range(1, l):
            divisor = k
            if not map:
                divisor = float(len(user_actual))
            # top k recommendations and top k actually rated movies for user
            user_rec = recommendations[user][1:k+1]
            user_actual_k = set(user_actual[:k])
            # only add if the kth item was relevant
            if user_rec[-1] in user_actual_k:
                # find intersections 
                user_rec = set(user_rec)
                intersection = list(user_rec & user_actual_k)
                # add precision or recall to sum
                sum += len(intersection)/divisor
        # divide by min(m - number of movies, N)
        sum /= float(l)
        map_k += sum

    # print final map@k value
    map_k /= float(num_users)
    return map_k

In [16]:
def mean_precision(recommendations, num_users, user_5_df, user_4_df): # mostly nikki's
    # http://sdsawtelle.github.io/blog/output/mean-average-precision-MAP-for-recommender-systems.html

    # for each user
    metric = 0
    for user in range(num_users):
        l = len(recommendations[user])
        user_actual = user_5_df[user_5_df['userId']==user]['movieId']
        if user_actual.empty:
            user_actual = user_4_df[user_4_df['userId']==user]['movieId']
        if user_actual.empty:
            continue

        if l == 0:
            continue

        user_actual = set(user_actual.values)
        # top k recommendations and top k actually rated movies for user
        user_rec = set(recommendations[user])
        # find intersections
        intersection = list(user_rec & user_actual)
        # add precision to sum
        metric += len(intersection) / float(l)

    metric /= float(num_users)
    return metric

In [17]:
# calculate MAP @ k
print("MAP @ k")
print(map_or_mark(recommendations, num_users, user_5_df, user_4_df, True))

# calculate MAR @ k
print("Mean Precision")
print(mean_precision(recommendations, num_users, user_5_df, user_4_df))

MAP @ k
0.00012295062472908003
Mean Precision
0.002479328990079953


In [18]:
# end timer
end_whole = time.perf_counter()
print(f"Finished in {(end - start)/60:0.4f} minutes")

Finished in 0.0839 minutes
